In [25]:
# At the very top of your notebook
%matplotlib notebook

In [26]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

class SAR_ADC:
    def __init__(self, bits=8, v_ref=3.3):
        self.bits = bits
        self.v_ref = v_ref

    def convert(self, analog_input):
        digital_output = 0
        steps = []

        for i in range(self.bits-1, -1, -1):
            trial = digital_output | (1 << i)
            trial_voltage = (trial / (2**self.bits)) * self.v_ref
            steps.append((trial, trial_voltage))

            if trial_voltage <= analog_input:
                digital_output = trial

        return digital_output, steps

    def animate(self, analog_input):
        digital_output, steps = self.convert(analog_input)

        fig, ax = plt.subplots()
        ax.set_title("SAR ADC Conversion Animation")
        ax.set_xlabel("Step")
        ax.set_ylabel("Voltage (V)")
        ax.set_ylim(0, self.v_ref)
        ax.axhline(y=analog_input, color='r', linestyle='-', label=f'Analog Input: {analog_input:.2f} V')
        ax.legend()
        ax.grid(True)

        xdata, ydata = [], []
        ln, = plt.plot([], [], 'bo--')
        text_annotations = []

        def init():
            ax.set_xlim(0, len(steps) + 1)
            return ln,

        def update(frame):
            xdata.append(frame + 1)
            ydata.append(steps[frame][1])
            ln.set_data(xdata, ydata)
            # Clear previous annotations
            for txt in text_annotations:
                txt.remove()
            text_annotations.clear()
            # Add new annotations
            for i, v in enumerate(ydata):
                text = ax.text(xdata[i], v + 0.05, f"{v:.2f} V", ha='center', color='blue')
                text_annotations.append(text)
            return ln,

        ani = animation.FuncAnimation(fig, update, frames=range(len(steps)),
                                      init_func=init, blit=False, interval=500, repeat=False)

        display(HTML(ani.to_jshtml()))

        print(f"Final Digital Output: {bin(digital_output)} ({digital_output})")



In [27]:
sar_adc = SAR_ADC(bits=8, v_ref=15)

In [28]:
analog_input = float(2)
sar_adc.animate(analog_input)


<IPython.core.display.Javascript object>

Final Digital Output: 0b100010 (34)
